In [31]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf

val sparkConf = new SparkConf()

val spark = SparkSession.builder()
            .config(sparkConf)
            .getOrCreate()

sparkConf = org.apache.spark.SparkConf@4d322957
spark = org.apache.spark.sql.SparkSession@6b02397b


lastException: Throwable = null


org.apache.spark.sql.SparkSession@6b02397b

In [32]:
val customers = spark.read
                .format("csv")
                .option("header", true)
                .option("inferSchema", true)
                .option("path", "/user/itv004483/sparkinput/customers.csv")
                .load()

val orders = spark.read
                .format("csv")
                .option("header", true)
                .option("inferSchema", true)
                .option("path", "/user/itv004483/sparkinput/orders.csv")
                .load()


customers = [customer_id: int, customer_fname: string ... 7 more fields]
orders = [order_id: int, order_date: timestamp ... 2 more fields]


[order_id: int, order_date: timestamp ... 2 more fields]

In [37]:
//find number of orders by each customer
import org.apache.spark.sql.functions.{col, sum, count}

val condition = orders.col("order_customer_id") === customers.col("customer_id")

val joinDF = orders.join(customers, condition, "inner")
                    .select("customer_id", "order_id")
                    .groupBy("customer_id")
                    .agg(count("order_id").alias("countOfOrders"))
                    .filter(col("countOfOrders") > 5)
                    .sort("customer_id")
                

joinDF.show


+-----------+-------------+
|customer_id|countOfOrders|
+-----------+-------------+
|          3|            7|
|          4|            6|
|          7|            8|
|          8|            8|
|          9|            6|
|         12|            8|
|         13|            6|
|         14|            9|
|         16|            7|
|         18|            8|
|         19|            9|
|         20|            7|
|         22|            6|
|         23|            8|
|         27|            7|
|         31|            6|
|         34|            7|
|         38|            6|
|         40|            7|
|         41|            6|
+-----------+-------------+
only showing top 20 rows



condition = (order_customer_id = customer_id)
joinDF = [customer_id: int, countOfOrders: bigint]


[customer_id: int, countOfOrders: bigint]

In [38]:

//spark sql

customers.createOrReplaceTempView("customers")
orders.createOrReplaceTempView("orders")

spark.sql("""
    select customer_id, count(order_id) as CountOfOrders 
    from customers c
    join orders o
        on c.customer_id = o.order_customer_id
    group by customer_id
    having CountOfOrders > 5
    order by customer_id
""").show

+-----------+-------------+
|customer_id|CountOfOrders|
+-----------+-------------+
|          3|            7|
|          4|            6|
|          7|            8|
|          8|            8|
|          9|            6|
|         12|            8|
|         13|            6|
|         14|            9|
|         16|            7|
|         18|            8|
|         19|            9|
|         20|            7|
|         22|            6|
|         23|            8|
|         27|            7|
|         31|            6|
|         34|            7|
|         38|            6|
|         40|            7|
|         41|            6|
+-----------+-------------+
only showing top 20 rows



In [41]:
spark.sql("select * from orders limit 5").show

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+

